In [ ]:
# ===================================
# MiceLight 1.0 2022/09/20 
# Yiyun Chen, Duo Zhang
# ====================================
# ========= Master Block =========
# ===== Package load-in =====
import warnings
import pandas as pd
warnings.simplefilter(action='ignore', category=FutureWarning) 
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
import numpy as np
import scipy
from scipy import signal
import matplotlib.pyplot as plt
from PIL import Image
import cv2 as cv
import os as os
import math
import matplotlib.cm as cm

# ===== Functions =====

# Read chosen folder
def fast_scandir(dirname):
    subfolders= [f.path for f in os.scandir(dirname) if f.is_dir()]
    for dirname in list(subfolders):
        subfolders.extend(fast_scandir(dirname))
    return subfolders

def get_immediate_subdirectories(a_dir):
    return [f.path for f in os.scandir(a_dir) if f.is_dir()]

#Read file
def ReadImage(ImgAbsPath):
  img_in = Image.open(ImgAbsPath)
  img_array = np.array(img_in)
  return img_array



In [ ]:
# ===== USER INPUT =====
# ******** Choose IVIS image folder here ********
#======================
# Folder structure:
# > Selected folder<  <-
#    |-A
#    | |- IVIS image folder 1
#    | |- IVIS image folder 2
#    |-B - ....
#   ...



FilePath = ""

#=====================
LuminScanMode = "CircularFit" # "BoundaryFit" or "CircularFit"


In [ ]:
# ===== MAIN BLOCK =====
Main_df = pd.DataFrame(columns = ['MouseCentX', 'MouseCentY', 'MouseContArea', 'HasTumor', 'SignalCentX', 'SignalCentY', 'SignalPxArea', 'TotalFlux'])
Flist = fast_scandir(FilePath) # Read folder content
RmList = get_immediate_subdirectories(FilePath)
Flist = [x for x in Flist if x not in RmList]

for ImageN in range(len(Flist)):
    # ========= Progress indicator =========
    print(str(ImageN+1)+ ' / ' +str(len(Flist)))
    print(Flist[ImageN])
    
    # ========= Metadata collection starts =========
    LuminArray = ReadImage(Flist[ImageN]+"/luminescent.TIF")
    PhotoArray = ReadImage(Flist[ImageN]+"/photograph.TIF")
    with open(Flist[ImageN]+"/ClickInfo.txt","r") as f:
        for l_no, line in enumerate(f):
            if 'Read Bias Level:' in line:
                BiasLevel = line.split("\t")
                BiasLevel = float(BiasLevel[1].strip())         
                break
            
    with open(Flist[ImageN]+"/ClickInfo.txt","r") as f:
        for l_no, line in enumerate(f):
            if '*** ClickNumber:' in line:
                ClickNumber = line.split("\t")
                ClickNumber = ClickNumber[1].strip()            
                break
            
    with open(Flist[ImageN]+"/ClickInfo.txt","r") as f:
        for l_no, line in enumerate(f):
            if 'Acquisition Date:' in line:
                AcquisitionDate = line.split("\t")
                AcquisitionDate = AcquisitionDate[1].strip()               
                break
            
    with open(Flist[ImageN]+"/ClickInfo.txt","r") as f:
        for l_no, line in enumerate(f):
            if 'Acquisition Time:' in line:
                AcquisitionTime = line.split("\t")
                AcquisitionTime = AcquisitionTime[1].strip()              
                break
            
    with open(Flist[ImageN]+"/AnalyzedClickInfo.txt","r") as f:
        for l_no, line in enumerate(f):
            if 'Experiment:' in line:
                Experiment = line.split("\t")
                Experiment = Experiment[1].strip()         
                break
            
    with open(Flist[ImageN]+"/AnalyzedClickInfo.txt","r") as f:
        for l_no, line in enumerate(f):
            if 'Comment1:' in line:
                Comment1 = line.split("\t")
                Comment1 = Comment1[1].strip()
                #print(Comment1)            
                break

    with open(Flist[ImageN]+"/AnalyzedClickInfo.txt","r") as f:
        for l_no, line in enumerate(f):
            if 'Comment2:' in line:
                Comment2 = line.split("\t")
                Comment2 = Comment2[1].strip()        
                break

    with open(Flist[ImageN]+"/ClickInfo.txt","r") as f:
        for l_no, line in enumerate(f):
            if 'Field of View:' in line:
                FOV = line.split("\t")
                FOV = float(FOV[1].strip())           
                break          

    # "C" or "D" - C= x510.1; D= x1486.15 for count to flux        

    if FOV > 22 and FOV < 23:
        ViewOption = "D"
    elif FOV > 13 and FOV < 14:
        ViewOption = "C"    
    LuminArray_copy = LuminArray - BiasLevel
    # ========= Metadata collection ends =========
    
    # ========= Process Photograph.TIF Starts =========
    # Create per photo dataframe
    Photo_df = pd.DataFrame(columns = ['MouseCentX', 'MouseCentY', 'MouseContArea'])
    # Read and process photo =====
    PhotoArray[PhotoArray > np.percentile(PhotoArray, 99)] = np.percentile(PhotoArray, 75)
    multiplier = np.percentile(PhotoArray, 100) / 255.0
    PhotoArray = ((PhotoArray / multiplier)).astype(np.uint8)

    PhotoArray = cv.resize(PhotoArray, dsize = LuminArray.shape, interpolation=cv.INTER_CUBIC) # Resize photo so size = luminescent




    # Count mice =====
    PhotoArray_inv = cv.bitwise_not(PhotoArray) # invert
    PhotoArray_inv = PhotoArray_inv[(PhotoArray.shape[0])//9:,:]  # cut top and bottom
    PhotoArray_inv = PhotoArray_inv[:-(PhotoArray_inv.shape[0])//9,:]

    PhotoArray_body = cv.convertScaleAbs(PhotoArray_inv, alpha = 1.2, beta = 0) #adjust alpha



    # Make mask
    PhotoArray_body[PhotoArray_body <= np.percentile(PhotoArray_body, 88)] = 0
    PhotoArray_body[PhotoArray_body > np.percentile(PhotoArray_body, 88)] = 255
    mask_body = np.zeros(PhotoArray_body.shape, np.uint8)

    # Make contours
    contours2, hier2 = cv.findContours(PhotoArray_body,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    numOfMouse = 0
    
    if ViewOption == "D":
        MouseSize = 400
    elif ViewOption == "C":
        MouseSize = 5000
    
    
    for cnt in contours2:
        if MouseSize < cv.contourArea(cnt):  # MAGIC NUMBER 500 for mouse size in 240*240
            ellipse = cv.fitEllipse(cnt)
            numOfMouse = numOfMouse + 1
            cv.drawContours(mask_body,[cnt],0,255,-1)
            cv.ellipse(mask_body, ellipse, (0,255,0),1)
            coord = ellipse[0]
            cv.circle(mask_body, (int(coord[0]),int(coord[1])), radius=0, color=(0, 0, 255), thickness=-1)
            mouse_row = {'MouseCentX':coord[0], 'MouseCentY':coord[1], 'MouseContArea':cv.contourArea(cnt)}
            Photo_df = Photo_df.append(mouse_row, ignore_index=True)


    plt.title('Mouse: ' + str(numOfMouse), fontsize=10, x=0.1)   
    
    plt.imsave(Flist[ImageN]+ '\\' + 'MouseCount.png', mask_body, cmap = 'gray')
    Photo_df = Photo_df.sort_values(by=['MouseCentX'])
    Photo_df = Photo_df.reset_index(drop=True)
    # ========= Process Photograph.TIF Ends =========
    
    # ========= Process Luminescent.TIF Starts =========
    # Create per Lumin dataframe
    Lumin_df = pd.DataFrame(columns = ['SignalCentX', 'SignalCentY', 'SignalPxArea', 'SignalSum'])

    # Read and process luminescent



    # Lumin Count
    if np.percentile(LuminArray_copy, 100)/100 <= 80:
        ret,thresh0 = cv.threshold(LuminArray_copy, 80, np.percentile(LuminArray_copy, 100),cv.THRESH_BINARY)
    elif np.percentile(LuminArray_copy, 100)/100 > 80:
        ret,thresh0 = cv.threshold(LuminArray_copy, np.percentile(LuminArray_copy, 100)/100, np.percentile(LuminArray_copy, 100),cv.THRESH_BINARY)
        
    thresh0 = thresh0.astype(np.uint8)


    thresh_draw = thresh0

    numOftumor = 0

    Tumorlist = []

    if LuminScanMode == "BoundaryFit":
        contours3, hier3 = cv.findContours(thresh_draw,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
        for cnt in contours3:
            if 0 < cv.contourArea(cnt): # MAGIC NUMBER for tumor size when 240*240
                thresh_draw = thresh0
                Tumorcont = cv.drawContours(thresh_draw, [cnt], 0, (255,255,255),1)
                numOftumor = numOftumor + 1
                ellipse = cv.minEnclosingCircle(cnt)
                coord = ellipse[0]
                Fill = cv.fillPoly(thresh_draw, pts = [cnt], color=(255,255,255))
                tumor_row = {'SignalCentX':coord[0], 'SignalCentY':coord[1]}
                Lumin_df = Lumin_df.append(tumor_row, ignore_index=True)
                pts = np.where(thresh_draw == 255)
                Tumorlist.append(pts)
    elif LuminScanMode == "CircularFit":
        contours3, hier3 = cv.findContours(thresh_draw,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
        for cnt in contours3:
            if 0 < cv.contourArea(cnt): # MAGIC NUMBER for tumor size when 240*240
                thresh_draw = thresh0
                Tumorcont = cv.drawContours(thresh_draw, [cnt], 0, (255,255,255),2)
                numOftumor = numOftumor + 1
                ellipse = cv.minEnclosingCircle(cnt)
                coord = ellipse[0]
                Fill = cv.fillPoly(thresh_draw, pts = [cnt], color=(255,255,255))
                CirC = cv.circle(thresh_draw, (int(coord[0]),int(coord[1])), radius=2, color=(255, 255, 255), thickness=2)
                tumor_row = {'SignalCentX':coord[0], 'SignalCentY':coord[1]}
                Lumin_df = Lumin_df.append(tumor_row, ignore_index=True)
                pts = np.where(thresh_draw == 255)
                Tumorlist.append(pts)


    plt.title("Tumor: " + str(numOftumor) ,fontsize=10, x=0.1)
    plt.imsave(Flist[ImageN]+ '\\' + 'SignalCount.png', thresh_draw, cmap = 'gray')


    # Parse tumorlist items to xy coord =====
    length = len(Tumorlist)
    for i in range(length):
        A = np.array([Tumorlist[i][0], Tumorlist[i][1]])
        A = A.transpose()
        A = tuple(map(tuple, A))
        Tumorlist[i] = A

    if length > 1:
        for i in reversed(range(1, length)):
              Tumorlist[i] = list(set(Tumorlist[i]) - set(Tumorlist[i-1]))
        Tumorlist[0] = list(set(Tumorlist[0]))
    elif length == 1:
        Tumorlist[0] = list(set(Tumorlist[0]))

    # xy coord to Lumin Sum list =====        
    TumorSizeList = []

    for t in range(length):
        TLength = len(Tumorlist[t])
        SignalSum = 0
        for pixel in range(TLength):
            if LuminArray[Tumorlist[t][pixel][0]][Tumorlist[t][pixel][1]] <= BiasLevel:
                SignalSum += 0
            elif LuminArray[Tumorlist[t][pixel][0]][Tumorlist[t][pixel][1]] > BiasLevel:
                SignalSum += LuminArray[Tumorlist[t][pixel][0]][Tumorlist[t][pixel][1]]-BiasLevel
        TumorSizeList.append(SignalSum)
        Lumin_df['SignalPxArea'][t] = TLength
        Lumin_df['SignalSum'][t] = SignalSum

    Lumin_df = Lumin_df.sort_values(by=['SignalCentX'])
    Lumin_df = Lumin_df.reset_index(drop=True)
    # ========= Process Luminescent.TIF Ends =========
    
    # ========= Create Merge Starts =========
    if np.percentile(LuminArray_copy, 100)/100 <= 80:
        masked_data = np.ma.masked_where(LuminArray_copy < 80, LuminArray)
    elif np.percentile(LuminArray_copy, 100)/100 > 80:
        masked_data = np.ma.masked_where(LuminArray_copy < np.percentile(LuminArray_copy, 100)/100, LuminArray)
        
    fig, ax = plt.subplots()

    # plot just the positive data and save the
    # color "mappable" object returned by ax1.imshow
    pos = ax.imshow(PhotoArray,'gray')

    # repeat everything above for the negative data
    # you can specify location, anchor and shrink the colorbar
    if ViewOption == "D":
        neg = ax.imshow(masked_data*1486.15, "rainbow", interpolation='none')
    elif ViewOption == "C":
        neg = ax.imshow(masked_data*510.1, "rainbow", interpolation='none')


    plt.axis('off')

    cbar = fig.colorbar(neg, ax=ax, location='right')
    cbar.set_label("Total Flux (p/s)")
    plt.suptitle(ClickNumber + ', ' + AcquisitionDate + ', ' + AcquisitionTime ,fontsize=8, y=0.1)
    plt.title('Mouse: ' + str(numOfMouse) + "   " + "Signal found: "+ str(numOftumor),fontsize=10, x=0.25)
    plt.savefig(Flist[ImageN]+ '\\' + 'Merged.png')
    # ========= Create Merge Ends =========
    
    # ========= Status Indicator =========
    print('Mouse: ' + str(numOfMouse) + "   " + "Signal found: "+ str(numOftumor))
    
    # ========= Create dataframe Starts =========
    Total_df = pd.DataFrame(columns = ['MouseCentX', 'MouseCentY', 'MouseContArea', 'HasTumor', 'SignalCentX', 'SignalCentY', 'SignalPxArea', 'TotalFlux'])

    for Pr in range(len(Photo_df)):
        ScanRangeLeft = Photo_df['MouseCentX'][Pr] - (LuminArray.shape[0]//11)
        ScanRangeRight = Photo_df['MouseCentX'][Pr] + (LuminArray.shape[0]//11)
        tumorarray = []
        for Tid in range(len(Lumin_df)):
            if Lumin_df["SignalCentX"][Tid] > ScanRangeLeft and Lumin_df["SignalCentX"][Tid] < ScanRangeRight:
                tumorarray.append(Tid)
        if len(tumorarray) > 1:
            Bigsize = Lumin_df.iloc[tumorarray].sort_values(by=['SignalPxArea'], ascending = False).iloc[0][2]
            for Tid in range(len(Lumin_df)):
                if Lumin_df["SignalPxArea"][Tid] == Bigsize:   
                    tumorarray = [Tid]

        if len(tumorarray) == 0:
            Total_row = {'MouseCentX':Photo_df['MouseCentX'][Pr], 'MouseCentY':Photo_df['MouseCentY'][Pr], 'MouseContArea':Photo_df['MouseContArea'][Pr], 'HasTumor':"N", 'SignalCentX': 0, 'SignalCentY':0, 'SignalPxArea':0, 'TotalFlux':0}
            Total_df = Total_df.append(Total_row, ignore_index=True)
        elif len(tumorarray) != 0:
            if ViewOption == "D":
                Total_row = {'MouseCentX':Photo_df['MouseCentX'][Pr], 'MouseCentY':Photo_df['MouseCentY'][Pr], 'MouseContArea':Photo_df['MouseContArea'][Pr], 'HasTumor':"Y", 'SignalCentX': Lumin_df['SignalCentX'][tumorarray[0]], 'SignalCentY':Lumin_df['SignalCentY'][tumorarray[0]], 'SignalPxArea':Lumin_df['SignalPxArea'][tumorarray[0]], 'TotalFlux':(Lumin_df['SignalSum'][tumorarray[0]])*1486.15}
            elif ViewOption == "C":
                Total_row = {'MouseCentX':Photo_df['MouseCentX'][Pr], 'MouseCentY':Photo_df['MouseCentY'][Pr], 'MouseContArea':Photo_df['MouseContArea'][Pr], 'HasTumor':"Y", 'SignalCentX': Lumin_df['SignalCentX'][tumorarray[0]], 'SignalCentY':Lumin_df['SignalCentY'][tumorarray[0]], 'SignalPxArea':Lumin_df['SignalPxArea'][tumorarray[0]], 'TotalFlux':(Lumin_df['SignalSum'][tumorarray[0]])*510.1}
            Total_df = Total_df.append(Total_row, ignore_index=True)

    Total_df.insert(0, "ClickNumber", [ClickNumber] * len(Total_df))
    Total_df.insert(1, "AcquisitionDate", [AcquisitionDate] * len(Total_df))
    Total_df.insert(2, "AcquisitionTime", [AcquisitionTime] * len(Total_df))
    Total_df.insert(3, "Experiment", [Experiment] * len(Total_df))
    Total_df.insert(4, "Comment1", [Comment1] * len(Total_df))
    Total_df.insert(5, "Comment2", [Comment2] * len(Total_df))
    Total_df.insert(6, "ViewOption", [ViewOption] * len(Total_df))
    Total_df.to_csv(Flist[ImageN]+ '\\' + 'Stats.csv', index=False)
    Main_df = pd.concat([Main_df, Total_df], ignore_index = True, sort = False)
    # ========= Create dataframe Ends =========
    
    
# ========= Save Main dataframe Main_df =========
Main_df.to_csv(FilePath+ '\\' + 'Stats.csv', index=False)